## Integrantes
Iván Fernandez París

Javier Fraile Iglesias

# APARTADO 1

In [2]:
from Datos import Datos
import EstrategiaParticionado
import ClasificadorNB

In [3]:
dataset_ttt = Datos('./tic-tac-toe.csv')
dataset_german = Datos('./german.csv')

In [4]:
# Creamos las estrategias de particionado para validacion simple y cruzada
estrategia_vs = EstrategiaParticionado.ValidacionSimple(5, 30)
estrategia_vc = EstrategiaParticionado.ValidacionCruzada(10)

In [5]:
# Inicializamos los clasificadores
clasificador = ClasificadorNB.ClasificadorNaiveBayes()
clasificador_laplace = ClasificadorNB.ClasificadorNaiveBayes(laplace=True)

In [7]:
errores_ttt_vs = clasificador.validacion(estrategia_vs, dataset_ttt)
errores_ttt_vc = clasificador.validacion(estrategia_vc, dataset_ttt)

errores_lp_ttt_vs = clasificador_laplace.validacion(estrategia_vs, dataset_ttt)
errores_lp_ttt_vc = clasificador_laplace.validacion(estrategia_vc, dataset_ttt)

errores_german_vs = clasificador.validacion(estrategia_vs, dataset_german)
errores_german_vc = clasificador.validacion(estrategia_vc, dataset_german)

errores_lp_german_vs = clasificador_laplace.validacion(estrategia_vs, dataset_german)
errores_lp_german_vc = clasificador_laplace.validacion(estrategia_vc, dataset_german)

### Errores (Nuestro Modelo)

|   Dataset   | Estrategia | Laplace | Media_Error | Desviacion Típica |
|-------------|------------|---------|-------------|-------------------|
| tic-tac-toe |  V.Simple  |    No   |  0.3076923  |     0.0119086     |
| tic-tac-toe |  V.Simple  |    Si   |  0.3076923  |     0.0119086     |
| tic-tac-toe | V.Cruzada  |    No   |  0.5611702  |     0.2263288     |
| tic-tac-toe | V.Cruzada  |    Si   |  0.5611702  |     0.2263288     |
|   german    |  V.Simple  |    No   |  0.2566433  |     0.0215084     |
|   german    |  V.Simple  |    Si   |  0.2566433  |     0.0215084     |
|   german    | V.Cruzada  |    No   |  0.2556515  |     0.0393679     |
|   german    | V.Cruzada  |    Si   |  0.2556515  |     0.0393679     |

### Analisis de los resultados

En cuanto a los resultados, en tic-tac-toe, los errores en la validacion cruzada y la simple, son mas distantes; mientras que en german, se mantienen parejos. 

Respecto al efecto de Lapalce, de la forma en la que lo hemos aplicado, no afecta a los resultados. Esto se debe a que la correccion afecta cuado un suceso no ocurre (probabilidad 0), pero en estos conjuntos de datos no se da esta condicion. En general, Laplace puede no afectar a la decision final del clasificador.

# Apartado 2

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB, GaussianNB, CategoricalNB

### Preparacion de datos

In [7]:
# Sobre nuestros datos german y ttt
data_ttt = pd.DataFrame(dataset_ttt.datos)
data_german = pd.DataFrame(dataset_german.datos)

# Obtenemos todos los atributos X y la clase y de tic-tac-toe.csv
X_ttt = data_ttt.iloc[:, :-1]
y_ttt = data_ttt.iloc[:, -1]

# Obtenemos todos los atributos X y la clase y de german.csv
X_german = data_german.iloc[:, :-1]
y_german = data_german.iloc[:, -1]

### Creación de modelos

In [8]:
# GaussianModel
gaussian_model = GaussianNB()

# MultinomialModel
multinomial_model = MultinomialNB(fit_prior=False)
multinomial_model_laplace = MultinomialNB(fit_prior=False, alpha=1)

# CategoricalModel
categorical_model = CategoricalNB(fit_prior=False)
categorical_model_laplace = CategoricalNB(fit_prior=False, alpha =1)

### Validaciones simple y cruzada con Sklearn

In [9]:
# Funcion auxiliar que devuelve la lista de errores para el modelo especifico (Validacion simple)
def errors_VS(X, y, size, n_ejec, model):
    errors = []
    errors_laplace = []
    
    for i in range(0, n_ejec):
        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size)        
        
        if model == "gaussian_model":
            # Se ajustan los datos y se anota el error (1 - acierto)
            gaussian_model.fit(X_train, y_train)
            errors.append(1 - gaussian_model.score(X_test, y_test))
        elif model == "multinomial_model":
            multinomial_model.fit(X_train, y_train)
            errors.append(1 - multinomial_model.score(X_test, y_test))
            multinomial_model_laplace.fit(X_train, y_train)
            errors_laplace.append(1 - multinomial_model_laplace.score(X_test, y_test))
        elif model == "categorical_model":
            categorical_model.fit(X_train, y_train)
            errors.append(1 - categorical_model.score(X_test, y_test))
            categorical_model_laplace.fit(X_train, y_train)
            errors_laplace.append(1 - categorical_model_laplace.score(X_test, y_test))
        else:
            return 0
        
    return errors, errors_laplace

In [10]:
# Funcion auxiliar que devuelve la lista de errores para el modelo especifico (Validacion Cruzada)

def errors_VC(X, y, folds, model):
    errors = []
    errors_laplace = []
    
    if model == "gaussian_model":
        errors = cross_val_score(gaussian_model, X, y, cv=folds)
    elif model == "multinomial_model":
        errors = cross_val_score(multinomial_model, X, y, cv=folds)
        errors_laplace = cross_val_score(multinomial_model_laplace, X, y, cv=folds)
    elif model == "categorical_model":
        errors = cross_val_score(categorical_model, X, y, cv=folds)
        errors_laplace = cross_val_score(categorical_model_laplace, X, y, cv=folds)
    else:
        return 0
    
    return errors, errors_laplace

#### GaussianNB

In [11]:
# Validacion simple con 5 ejecuciones para GaussianNB

# 30% de test y 5 iteraciones sobre tic-tac-toe
errors_ttt, _ = errors_VS(X_ttt, y_ttt, 0.3, 10, "gaussian_model")

# 30% de test y 5 iteraciones sobre tic-tac-toe
errors_german, _ = errors_VS(X_german, y_german, 0.3, 10, "gaussian_model")

In [12]:
# Validacion cruzada con 10 particiones para GaussianNB

# 10 particiones para tic-tac-toe
errors_ttt, _ = errors_VC(X_ttt, y_ttt, 10, "gaussian_model")

# 10 particiones para german
errors_german, _ = errors_VC(X_german, y_german, 10, "gaussian_model")

#### MultinomialNB

In [13]:
# Validacion simple con 5 ejecuciones para MultinomialNB

# 30% de test y 5 iteraciones sobre tic-tac-toe
errors_ttt, errors_ttt_laplace = errors_VS(X_ttt, y_ttt, 0.3, 5, "multinomial_model")

# 30% de test y 5 iteraciones sobre tic-tac-toe
errors_german, errors_german_laplace = errors_VS(X_german, y_german, 0.3, 5, "multinomial_model")

In [14]:
# Validacion simple con 5 ejecuciones para MultinomialNB

# 10 particiones para tic-tac-toe
errors_ttt, errors_ttt_laplace = errors_VC(X_ttt, y_ttt, 10, "multinomial_model")

# 10 particiones para german
errors_german, errors_german_laplace = errors_VC(X_german, y_german, 10, "multinomial_model")

#### CategoricalNB

In [15]:
# Validacion simple con 5 ejecuciones para CategoricalNB

# 30% de test y 5 iteraciones sobre tic-tac-toe
errors_ttt, errors_ttt_laplace = errors_VS(X_ttt, y_ttt, 0.3, 5, "categorical_model")

In [16]:
# Validacion cruzada con 10 particiones para CategoricalNB

# 10 particiones para tic-tac-toe
errors_ttt, errors_ttt_laplace = errors_VC(X_ttt, y_ttt, 10, "categorical_model")

### Errores (Con Sklearn)

|   Dataset   | Estrategia | Laplace | Gaussian(mean) | Gaussian(std) |MultinomialNB(mean)|MultinomialNB(std)|CategoricalNB(mean)| CategoricalNB(std)|
|-------------|------------|---------|----------------|---------------|-------------------|------------------|-------------------|-------------------|
| tic-tac-toe |  V.Simple  |    No   |   0.2958333    |   0.0123055   |   0.4104166       |   0.0217838      |   0.3236111       |   0.0341903       |
| tic-tac-toe |  V.Simple  |    Si   |   0.2958333    |   0.0123055   |   0.4104166       |   0.0217838      |   0.3236111       |   0.0341903   |
| tic-tac-toe | V.Cruzada  |    No   |   0.6858662    |   0.0332495   |   0.5322587       |   0.0956273      |   0.6034978       |   0.1093299   |
| tic-tac-toe | V.Cruzada  |    Si   |   0.6858662    |   0.0332495   |   0.5322587       |   0.0956273      |   0.6034978       |   0.1093299   |
|   german    |  V.Simple  |    No   |   0.2686666    |   0.0225487   |   0.3440000       |   0.0180616      |   ---------       |   ---------   |
|   german    |  V.Simple  |    Si   |   0.2686666    |   0.0225487   |   0.3440000       |   0.0180616      |   ---------       |   ---------   |
|   german    | V.Cruzada  |    No   |   0.7340000    |   0.0431740   |   0.6270000       |   0.0661891      |   ---------       |   ---------   |
|   german    | V.Cruzada  |    Si   |   0.7340000    |   0.0431740   |   0.6270000       |   0.0661891      |   ---------       |   ---------   |

### Discursión de Resultados

Descartamos en ambos modelos el clasificador MultinomialNB, puesto que es la version Naive Bayes para datos distribuidos multinomialmente, es decir, es una generalizacion de la distribucion binomial cuando el experimento no tiene solo dos resultados posibles, si no, tres o mas (en nuestro caso son dos psibilidades de resultados).

Para el conjunto de datos tic-tac-toe, creemos que el mejor modelo es el categoricalNB, ya que dicho modelo es la version Naive Bayes para datos categoricos (todos los atributos son categoricos), y cada atributo sigue su distribucion categorica.

Para el conjunto german, el mejor debe ser el gaussian porque descartamos Multinomial, y categorical no se puede ejecutar ya que no todos los atributos son categoricos.

## Preparacion de datos (One Hot Encoder)

In [17]:
# Se aplica oneHotEncoder a todos las filas del dataset, menos a la clase
dataframe = pd.read_csv('./tic-tac-toe.csv')

# Obtenemos todos los atributos X y la clase y de tic-tac-toe.csv
X_ttt = dataframe.iloc[:, :-1]
y_ttt = dataframe.iloc[:, -1]

# Aplicamos oneHotEncoder para los datos
nominal_categories = np.asarray(X_ttt)
encoder = OneHotEncoder(sparse=False, dtype=np.int32)
onehot = encoder.fit(nominal_categories)
X_ttt = encoder.transform(nominal_categories)

#### GaussianNB

In [18]:
# Validacion simple con 5 ejecuciones para GaussianNB

# 30% de test y 5 iteraciones sobre tic-tac-toe
errors_ttt, _ = errors_VS(X_ttt, y_ttt, 0.3, 5, "gaussian_model")

In [19]:
# Validacion cruzada con 10 particiones para GaussianNB

# 10 particiones para tic-tac-toe
errors_ttt, _ = errors_VC(X_ttt, y_ttt, 10, "gaussian_model")

#### MultinomialNB

In [20]:
# Validacion simple con 5 ejecuciones para MultinomialNB

# 30% de test y 5 iteraciones sobre tic-tac-toe
errors_ttt, errors_ttt_laplace = errors_VS(X_ttt, y_ttt, 0.3, 5, "multinomial_model")

In [21]:
# Validacion simple con 5 ejecuciones para MultinomialNB

# 10 particiones para tic-tac-toe
errors_ttt, errors_ttt_laplace = errors_VC(X_ttt, y_ttt, 10, "multinomial_model")

#### CategoricalNB

In [22]:
# Validacion simple con 5 ejecuciones para CategoricalNB

# 30% de test y 5 iteraciones sobre tic-tac-toe
errors_ttt, errors_ttt_laplace = errors_VS(X_ttt, y_ttt, 0.3, 5, "categorical_model")

In [23]:
# Validacion cruzada con 10 particiones para CategoricalNB

# 10 particiones para tic-tac-toe
errors_ttt, errors_ttt_laplace = errors_VC(X_ttt, y_ttt, 10, "categorical_model")

### Errores (One Hot Encoder)

|     Clasificador    | Estrategia | Laplace | Media_Error | Desviacion Típica |
|---------------|------------|---------|-------------|-------------------|
|  GaussianNB   |  V.Simple  |    No   |  0.3173611  |     0.0268598     |
|  GaussianNB   | V.Cruzada  |    No   |  0.6285416  |     0.0914832     |
| MultinomialNB |  V.Simple  |  Si/No  |  0.3375000  |     0.0172076     |
| MultinomialNB | V.Cruzada  |  Si/No  |  0.6045394  |     0.1085844     |
| CategoricalNB |  V.Simple  |  Si/No  |  0.3368055  |     0.0221787     |
| CategoricalNB | V.Cruzada  |  Si/No  |  0.6108442  |     0.1090512     |

### Comparación de Resultados

La codificacion utilizada en nuestro modelo es similar a labelEoncoder, es decir, se van asignando una secuencia de valores a cada atributo categorico. El problema, es que el modelo puede ver relaciones entre datos donde no las hay.

Para evitar este sesgo es frecuente realizar una codificación binaria a través de la clase OneHotEncoder.
Esta codificacion es muy util en el modelos de regresion lineal, pero no es nestro caso, y es por eso que los resultados obtenidos para cada clasificador, son muy similares a los anteriores.

# Apartado 3

|   Dataset   | Estrategia | Laplace | Gaussian(mean) | Gaussian(std) |MultinomialNB(mean)|MultinomialNB(std)|CategoricalNB(mean)| CategoricalNB(std)| ClasificadorNB(mean) | ClasificadorNB(std)|
|-------------|------------|---------|----------------|---------------|-------------------|------------------|-------------------|-------------------|----------------------|-------------------|
| tic-tac-toe |  V.Simple  |    No   |   0.2958333    |   0.0123055   |   0.4104166       |   0.0217838      |   0.3236111       |   0.0341903       |  0.3076923           |     0.0119086     |
| tic-tac-toe |  V.Simple  |    Si   |   0.2958333    |   0.0123055   |   0.4104166       |   0.0217838      |   0.3236111       |   0.0341903       |  0.3076923           |     0.0119086     |
| tic-tac-toe | V.Cruzada  |    Si   |   0.6858662    |   0.0332495   |   0.5322587       |   0.0956273      |   0.6034978       |   0.1093299       |  0.5611702           |     0.2263288     |
| tic-tac-toe | V.Cruzada  |    No   |   0.6858662    |   0.0332495   |   0.5322587       |   0.0956273      |   0.6034978       |   0.1093299       |  0.5611702           |     0.2263288     |
|   german    |  V.Simple  |    No   |   0.2686666    |   0.0225487   |   0.3440000       |   0.0180616      |   ---------       |   ---------       |  0.2566433           |     0.0215084     |
|   german    |  V.Simple  |    Si   |   0.2686666    |   0.0225487   |   0.3440000       |   0.0180616      |   ---------       |   ---------       |  0.2566433           |     0.0215084     |
|   german    | V.Cruzada  |    No   |   0.7340000    |   0.0431740   |   0.6270000       |   0.0661891      |   ---------       |   ---------       |  0.2556515           |     0.0393679     |
|   german    | V.Cruzada  |    Si   |   0.7340000    |   0.0431740   |   0.6270000       |   0.0661891      |   ---------       |   ---------       |  0.2556515           |     0.0393679     |

### Analisis de los resultados

Para el dataset de tic-tac-toe, cuando se realiza una validacion simple, los resultados obtenidos por cada clasificador son muy similares, destacando que el Multinomial da los peores resultados. Para la validacion cruzada, el Gaussian es el que peor prediccion realiza.

En el caso del conjunto de datos german, la validacion simple la tendencia es muy similar, obteniendose errores parejos, y en la validacion cruzada, destacar que nuestro calsificador tiene una tasa de error muy baja en comparacion con el resto de clasificadores.